Linear Regression

3.5 Lab : Linear Regression

In [1]:
# import library
import numpy as np
import pandas as pd

import statsmodels.api as sm

# import module from sm
from statsmodels.stats.outliers_influence import variance_inflation_factor as VIF 
from statsmodels.stats.anova import anova_lm

In [2]:
# import some function from ISLP
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize, poly)

We will built simple linear regression with ISLP models, and with boston dataset

In [3]:
# load dataset
boston = load_data('Boston')

# see the column
boston.columns

Index(['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax',
       'ptratio', 'lstat', 'medv'],
      dtype='object')

We start using sm.OLS() function to fit simple linear regression model. medv will be a respon dan lstst will be the single predictor

In [4]:
# create dataframe for data we will use
# predictor
X = pd.DataFrame({'intercept' : np.ones(boston.shape[0]),
                  'lstat' : boston['lstat']})

# response
y = boston['medv']

In [5]:
# review x
X

,intercept,lstat
0,1.0,4.98
1,1.0,9.14
2,1.0,4.03
3,1.0,2.94
4,1.0,5.33
...,...,...
501,1.0,9.67
502,1.0,9.08
503,1.0,5.64
504,1.0,6.48


In [6]:
# reviw y
y

0      24.0
1      21.6
2      34.7
3      33.4
4      36.2
       ... 
501    22.4
502    20.6
503    23.9
504    22.0
505    11.9
Name: medv, Length: 506, dtype: float64

In [7]:
# fit the model
# use sm.OLS()
model = sm.OLS(y, X) # specifies the model 
results = model.fit() # fits the model

In [8]:
# see the summary
summarize(results)

,coef,std err,t,P>|t|
intercept,34.5538,0.563,61.415,0.0
lstat,-0.9500,0.039,-24.528,0.0


In [9]:
# transfor the x/predictors
# use model spec as ms
design = MS(['lstat'])

# these two step can be combined with fit_transform
design = design.fit(boston)
X = design.transform(boston)
X[:4]

,intercept,lstat
0,1.0,4.98
1,1.0,9.14
2,1.0,4.03
3,1.0,2.94


In [10]:
# back to our fitted model
# we can see the fitted coef with
results.params

intercept    34.553841
lstat        -0.950049
dtype: float64

In [11]:
# predictions
# get_prediction() to make prediction
new_df = pd.DataFrame({'lstat' : [5, 10, 15]})
newX = design.transform(new_df)
newX

,intercept,lstat
0,1.0,5
1,1.0,10
2,1.0,15


In [12]:
# make prediction
new_predictions = results.get_prediction(newX)
new_predictions.predicted_mean

array([29.80359411, 25.05334734, 20.30310057])

In [13]:
# confidence interval of predicted values
new_predictions.conf_int(obs=True, alpha=0.05) # set alpha to 0.05 or 95% confidence interval

array([[17.56567478, 42.04151344],
       [12.82762635, 37.27906833],
       [ 8.0777421 , 32.52845905]])

3.6.3 Multiple Linear Regression

In [14]:
# now the predictor is lstat and age, the respon is medv
# use model spec again to create the design matrix
X = MS(['lstat', 'age']).fit_transform(boston)

# specify the model
model1 = sm.OLS(y, X)

# fitted the model
results1 = model1.fit()

# see the summary
summarize(results1)

,coef,std err,t,P>|t|
intercept,33.2228,0.731,45.458,0.000
lstat,-1.0321,0.048,-21.416,0.000
age,0.0345,0.012,2.826,0.005


In [15]:
# try to use all the variabel we have in boston to do regression
# first we drop the medv
terms = boston.columns.drop('medv')
terms

Index(['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax',
       'ptratio', 'lstat'],
      dtype='object')

In [16]:
# and we create matrix
X = MS(terms).fit_transform(boston)

# specify the model
model = sm.OLS(y, X)

# fit the model
results = model.fit()

# see the summary
summarize(results)

,coef,std err,t,P>|t|
intercept,41.6173,4.936,8.431,0.000
crim,-0.1214,0.033,-3.678,0.000
zn,0.0470,0.014,3.384,0.001
indus,0.0135,0.062,0.217,0.829
chas,2.8400,0.870,3.264,0.001
nox,-18.7580,3.851,-4.870,0.000
rm,3.6581,0.420,8.705,0.000
age,0.0036,0.013,0.271,0.787
dis,-1.4908,0.202,-7.394,0.000
rad,0.2894,0.067,4.325,0.000


in the output of regression (in case we use all of variabel) the p_value of age is high. so we may run a regression excluding that.

In [17]:
# let frop medv as y, and age
minus_age = boston.columns.drop(['medv', 'age'])

# create the matrix
Xma = MS(minus_age).fit_transform(boston)

# specify the model
model1 = sm.OLS(y, Xma)

# fit the model
results1 = model1.fit()

# see the summary
summarize(results1)

,coef,std err,t,P>|t|
intercept,41.5251,4.920,8.441,0.000
crim,-0.1214,0.033,-3.683,0.000
zn,0.0465,0.014,3.379,0.001
indus,0.0135,0.062,0.217,0.829
chas,2.8528,0.868,3.287,0.001
nox,-18.4851,3.714,-4.978,0.000
rm,3.6811,0.411,8.951,0.000
dis,-1.5068,0.193,-7.825,0.000
rad,0.2879,0.067,4.322,0.000
tax,-0.0127,0.004,-3.333,0.001


3.6.4 Multivariate Goodness of Fit

VIF (varianve inflation factor) are sometimes useful to assess the effect of collinearity

In [18]:
# see the VIF in model1 with list comprehension
vals = [VIF(X, i) for i in range(1, X.shape[1])]

# dataframe
vif =  pd.DataFrame({'vif' : vals}, index=X.columns[1:])

vif

,vif
crim,1.767486
zn,2.298459
indus,3.987181
chas,1.071168
nox,4.369093
rm,1.912532
age,3.088232
dis,3.954037
rad,7.445301
tax,9.002158


absolutely we can use VIF() xD

3.6.6 Non-Linear Transformations of the Predictors

polynomial transformation

In [19]:
X = MS([poly('lstat', degree=2), 'age']).fit_transform(boston)

# specify the model
model3 = sm.OLS(y, X)

# fit the model
results3 = model3.fit()

# see the summary
summarize(results3)

,coef,std err,t,P>|t|
intercept,17.7151,0.781,22.681,0.0
"poly(lstat, degree=2)[0]",-179.2279,6.733,-26.620,0.0
"poly(lstat, degree=2)[1]",72.9908,5.482,13.315,0.0
age,0.0703,0.011,6.471,0.0


In [20]:
anova_lm(results1, results3)

,df_resid,ssr,df_diff,ss_diff,F,Pr(>F)
0,494.0,11351.108037,0.0,NaN,NaN,NaN
1,502.0,14165.613251,-8.0,-2814.505214,12.46753,NaN


3.6.7 Qualitative Predictor

In [21]:
# load data
carseats = load_data('Carseats')
carseats.columns

Index(['Sales', 'CompPrice', 'Income', 'Advertising', 'Population', 'Price',
       'ShelveLoc', 'Age', 'Education', 'Urban', 'US'],
      dtype='object')

In [22]:
# create X and Y
allvars = list(carseats.columns.drop(['Sales']))
y = carseats['Sales']

# create dummy variabel for 'ShelveLoc
final = allvars + [('Income', 'Advertising'),
                   ('Price', 'Age')]

X = MS(final).fit_transform(carseats)

model = sm.OLS(y, X)

summarize(model.fit())

,coef,std err,t,P>|t|
intercept,6.5756,1.009,6.519,0.000
CompPrice,0.0929,0.004,22.567,0.000
Income,0.0109,0.003,4.183,0.000
Advertising,0.0702,0.023,3.107,0.002
Population,0.0002,0.000,0.433,0.665
Price,-0.1008,0.007,-13.549,0.000
ShelveLoc[Good],4.8487,0.153,31.724,0.000
ShelveLoc[Medium],1.9533,0.126,15.531,0.000
Age,-0.0579,0.016,-3.633,0.000
Education,-0.0209,0.020,-1.063,0.288
